In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [ ]:
df = pd.read_csv("diabetes.csv")

In [ ]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
X = df.drop("Outcome",axis=1)
y = df["Outcome"]

In [ ]:
sc = StandardScaler()

In [ ]:
X = sc.fit_transform(X)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
model = Sequential()

model.add(Dense(32,activation="relu",input_dim=8))
model.add(Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy",optimizer="Adam",metrics=["accuracy"])

In [ ]:
history = model.fit(X_train,y_train,epochs=100,batch_size=32,validation_split=0.2)

Epoch 1/100
16/16 [==============================] - 1s 13ms/step - loss: 0.7026 - accuracy: 0.5193 - val_loss: 0.6681 - val_accuracy: 0.5935
Epoch 2/100
16/16 [==============================] - 0s 4ms/step - loss: 0.6433 - accuracy: 0.6864 - val_loss: 0.6234 - val_accuracy: 0.7073
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5994 - accuracy: 0.7026 - val_loss: 0.5902 - val_accuracy: 0.7236
Epoch 4/100
16/16 [==============================] - 0s 7ms/step - loss: 0.5686 - accuracy: 0.7332 - val_loss: 0.5662 - val_accuracy: 0.7480
Epoch 5/100
16/16 [==============================] - 0s 7ms/step - loss: 0.5469 - accuracy: 0.7454 - val_loss: 0.5474 - val_accuracy: 0.7561
Epoch 6/100
16/16 [==============================] - 0s 6ms/step - loss: 0.5303 - accuracy: 0.7495 - val_loss: 0.5318 - val_accuracy: 0.7480
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5158 - accuracy: 0.7576 - val_loss: 0.5204 - val_accuracy: 0.7642
Epoch 8/100


#### 1. How to select appropriate optimizer
#### 2. No. of neurons in a layer
#### 3. How to select no. of hidden layers
#### 4. All in one model

In [ ]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import kerastuner as kt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [ ]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation="relu",input_dim=8))
  model.add(Dense(1,activation="sigmoid"))

  model.compile(optimizer=hp.Choice("optimizer",values=["adam","sgd","rmsprop","adadelta"]),loss="binary_crossentropy",metrics=["accuracy"])
  return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective="val_accuracy",
                        max_trials=5)

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,batch_size=32,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 18ms/step - loss: 0.5388 - accuracy: 0.7541 - val_loss: 0.5268 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5222 - accuracy: 0.7590 - val_loss: 0.5110 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5090 - accuracy: 0.7622 - val_loss: 0.4994 - val_accuracy: 0.8052
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4992 - accuracy: 0.7622 - val_loss: 0.4902 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4912 - accuracy: 0.7736 - val_loss: 0.4821 - val_accuracy: 0.8052
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4846 - accuracy: 0.7671 - val_loss: 0.4754 - val_accuracy: 0.7987
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4790 - accuracy: 0.7736 - val_loss: 0.4715 - val_accuracy: 0.7922
Epoch 14

### **To find no. of neurons**

In [ ]:
def build_model(hp):
  model = Sequential()

  units = hp.Int("units",8,128,step=8)
  model.add(Dense(units=units,activation="relu",input_dim=8))
  model.add(Dense(1,activation="sigmoid"))

  model.compile(optimizer="rmsprop",loss="binary_crossentropy",metrics=["accuracy"])

  return model 

In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective="val_accuracy",
                        max_trials=5,
                        directory = "new_dir",
                        project_name="tuner")

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 08s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'units': 104}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 0s 10ms/step - loss: 0.3958 - accuracy: 0.8176 - val_loss: 0.4553 - val_accuracy: 0.8182
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3945 - accuracy: 0.8208 - val_loss: 0.4564 - val_accuracy: 0.8117
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3939 - accuracy: 0.8257 - val_loss: 0.4537 - val_accuracy: 0.8182
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3941 - accuracy: 0.8208 - val_loss: 0.4550 - val_accuracy: 0.8117
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3931 - accuracy: 0.8306 - val_loss: 0.4556 - val_accuracy: 0.8117
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3929 - accuracy: 0.8241 - val_loss: 0.4574 - val_accuracy: 0.8312
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3927 - accuracy: 0.8257 - val_loss: 0.4568 - val_accuracy: 0.8117
Epoch 14

## Select no. of hidden layers

In [ ]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(72,activation="relu",input_dim=8))
  for i in range(hp.Int("num_layers",min_value=1,max_value=10)):
    model.add(Dense(72,activation="relu"))

  model.add(Dense(1,activation="sigmoid"))
  model.compile(optimizer="rmsprop",loss="binary_crossentropy",metrics=["accuracy"])

  return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective="val_accuracy",
                        max_trials=3,
                        directory="new_dir1",
                        project_name="num_layers")

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 06s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 14ms/step - loss: 0.4353 - accuracy: 0.7915 - val_loss: 0.4881 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4294 - accuracy: 0.8192 - val_loss: 0.5060 - val_accuracy: 0.7468
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4152 - accuracy: 0.8192 - val_loss: 0.4688 - val_accuracy: 0.7987
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4001 - accuracy: 0.8160 - val_loss: 0.5519 - val_accuracy: 0.7532
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4119 - accuracy: 0.8192 - val_loss: 0.5197 - val_accuracy: 0.7532
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.3772 - accuracy: 0.8420 - val_loss: 0.4775 - val_accuracy: 0.7922
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.3661 - accuracy: 0.8420 - val_loss: 0.5892 - val_accuracy: 0.7597
Epoch 14

## All in one

In [ ]:
def build_model(hp):
  model = Sequential()

  counter = 0

  for i in range(hp.Int("num_layers",min_value=1,max_value=10)):
    if counter == 0:
      model.add(Dense(
          hp.Int("units"+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice("activation"+str(i),values=["relu","tanh","sigmoid"])
      ))
    else:
      model.add(Dense(
          hp.Int("units"+str(i),min_value=8,max_value=128,step=8)
      ,activation=hp.Choice("activation"+str(i),values=["relu","tanh","sigmoid"])))

    counter+=1
  model.add(Dense(1,activation="sigmoid"))
  model.compile(optimizer=hp.Choice("optimizer",values=["adam","sgd","rmsprop","nadam","adadelta"]),loss="binary_crossentropy",metrics=["accuracy"])
  return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective="val_accuracy",
                        max_trials=3,
                        directory="new_dir2",
                        project_name="project1")

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 05s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 16,
 'activation0': 'tanh',
 'optimizer': 'adam',
 'units1': 96,
 'activation1': 'tanh',
 'units2': 72,
 'activation2': 'tanh',
 'units3': 48,
 'activation3': 'relu',
 'units4': 56,
 'activation4': 'tanh'}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train,epochs=100,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/100
20/20 [==============================] - 1s 11ms/step - loss: 0.4787 - accuracy: 0.7557 - val_loss: 0.4532 - val_accuracy: 0.8052
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4779 - accuracy: 0.7638 - val_loss: 0.4628 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4753 - accuracy: 0.7638 - val_loss: 0.4543 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4734 - accuracy: 0.7638 - val_loss: 0.4571 - val_accuracy: 0.7922
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4697 - accuracy: 0.7655 - val_loss: 0.4527 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4696 - accuracy: 0.7638 - val_loss: 0.4575 - val_accuracy: 0.7922
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4695 - accuracy: 0.7590 - val_loss: 0.4572 - val_accuracy: 0.7922
Epoch 13/

In [ ]:
def build_model(hp):
  model = Sequential()

  counter = 0

  for i in range(hp.Int("num_layers",min_value=1,max_value=10)):
    if counter == 0:
      model.add(Dense(
          hp.Int("units"+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice("activation"+str(i),values=["relu","tanh","sigmoid"])
      ))
      model.add(Dropout(hp.Choice("Dropout"+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(
          hp.Int("units"+str(i),min_value=8,max_value=128,step=8)
      ,activation=hp.Choice("activation"+str(i),values=["relu","tanh","sigmoid"])))
      model.add(Dropout(hp.Choice("Dropout"+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    counter+=1
  model.add(Dense(1,activation="sigmoid"))
  model.compile(optimizer=hp.Choice("optimizer",values=["adam","sgd","rmsprop","nadam","adadelta"]),loss="binary_crossentropy",metrics=["accuracy"])
  return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective="val_accuracy",
                        max_trials=3,
                        directory="new_dir2",
                        project_name="project1")

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6558441519737244
Total elapsed time: 00h 00m 08s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5,
 'units0': 8,
 'activation0': 'relu',
 'Dropout0': 0.7,
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'relu',
 'Dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'Dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'Dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'Dropout4': 0.1}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train,epochs=100,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/100
20/20 [==============================] - 1s 12ms/step - loss: 0.6497 - accuracy: 0.6336 - val_loss: 0.6155 - val_accuracy: 0.6429
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6349 - accuracy: 0.6580 - val_loss: 0.5932 - val_accuracy: 0.6429
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6304 - accuracy: 0.6417 - val_loss: 0.5809 - val_accuracy: 0.6429
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6115 - accuracy: 0.6596 - val_loss: 0.5741 - val_accuracy: 0.6429
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6146 - accuracy: 0.6596 - val_loss: 0.5711 - val_accuracy: 0.6429
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6067 - accuracy: 0.6482 - val_loss: 0.5680 - val_accuracy: 0.6429
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6197 - accuracy: 0.6596 - val_loss: 0.5688 - val_accuracy: 0.6429
Epoch 13/